In [ ]:
import numpy as np
import matplotlib.pylab as plt
import pandas as pd
import os
import math
import json
import re

from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_extraction import DictVectorizer
from sklearn.feature_selection import RFECV
from sklearn.linear_model import LinearRegression
from sklearn import tree
from sklearn.svm import LinearSVC
from sklearn.feature_selection import SelectFromModel
np.set_printoptions(suppress=True)

from sklearn.model_selection import train_test_split
import pandas as pd
from pandas import DataFrame
import matplotlib.pyplot as plt
import datetime
import torch
import torch.nn as nn
import numpy as np
from torch.utils.data import Dataset, DataLoader
import tensorflow_addons as tfa
from keras.models import Sequential, Input, Model
from keras_self_attention import SeqSelfAttention
from keras.layers.core import Dense, Dropout, Activation
from keras.layers import concatenate
from keras.layers import Bidirectional

from keras.layers.recurrent import LSTM,GRU
from keras.models import load_model
import keras
from tcn import TCN, tcn_full_summary
from keras_multi_head import MultiHead
from keras_multi_head import MultiHeadAttention
from keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.python.client import device_lib
from keras import backend as K
import tensorflow as tf
np.set_printoptions(threshold=np.inf)

import tensorflow_hub as hub
import nltk
from nltk.corpus import stopwords
from nltk.tokenize.treebank import TreebankWordDetokenizer
#from keras.preprocessing.sequence import pad_sequences

from spektral.layers import GCNConv
from spektral.models.gcn import GCN
#from transformers import BertTokenizer, TFBertModel, BertConfig
#nltk.download('stopwords')
embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")
from flair.models import TextClassifier
from flair.data import Sentence
classifier = TextClassifier.load('en-sentiment')
#embed = TFBertModel.from_pretrained("bert-base-uncased")
#stop_words = set(stopwords.words('english'))

In [ ]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

In [ ]:
X = pd.read_csv(os.getcwd()+'/price/alltrainings.csv')
X_test = pd.read_csv(os.getcwd()+'/price/alltestings.csv')
X_val = pd.read_csv(os.getcwd()+'/price/allvalidations.csv')
#X = X.drop(X.columns[0], axis=1)
#X_test = X_test.drop(X_test.columns[0], axis=1)
#X_val = X_val.drop(X_val.columns[0], axis=1)
y = X['label']
y_test = X_test['label']
y_val = X_val['label']
#baseline1
#X = X[['date','adj close','high','low','label']]
#X_val = X_val[['date','adj close','high','low','label']]
#X_test = X_test[['date','adj close','high','low','label']]

#baseline3 L1 regularization
#X = X[['date','adj close','high','low','open','close','volume','volume_nvi','trend_adx_pos','trend_mass_index','trend_cci','trend_aroon_up','trend_aroon_down','trend_aroon_ind','momentum_rsi','momentum_stoch','momentum_wr','RSI','Stoch','stoch_signal','TSI','WilliamsR','adx_neg','aroon_down','aroon','aroon_up','MI','CR','label']]
#X_test = X_test[['date','adj close','high','low','open','close','volume','volume_nvi','trend_adx_pos','trend_mass_index','trend_cci','trend_aroon_up','trend_aroon_down','trend_aroon_ind','momentum_rsi','momentum_stoch','momentum_wr','RSI','Stoch','stoch_signal','TSI','WilliamsR','adx_neg','aroon_down','aroon','aroon_up','MI','CR','label']]

#X = X[['date','adj close','high','low','open','close','volume','volume_nvi', 'volatility_atr', 'volatility_bbw', 'volatility_dchi', 'trend_macd', 'trend_macd_signal', 'trend_adx', 'trend_adx_pos', 'trend_adx_neg', 'trend_mass_index', 'trend_cci', 'trend_dpo', 'trend_kst_diff', 'trend_aroon_up', 'momentum_rsi', 'momentum_mfi', 'momentum_uo', 'momentum_stoch_signal', 'momentum_wr', 'momentum_ao', 'momentum_roc', 'others_dlr', 'others_cr', 'AO', 'MFI', 'ROC', 'RSI', 'stoch_signal', 'UO', 'WilliamsR', 'NVI', 'ATR', 'bb_bbw', 'dc_dchi', 'adx', 'adx_neg', 'aroon_up', 'CCI', 'DPO', 'KST', 'KST_diff', 'macd', 'macd_sig', 'MI', 'CR', 'DLR','label']]
#X_val = X_val[['date','adj close','high','low','open','close','volume','volume_nvi', 'volatility_atr', 'volatility_bbw', 'volatility_dchi', 'trend_macd', 'trend_macd_signal', 'trend_adx', 'trend_adx_pos', 'trend_adx_neg', 'trend_mass_index', 'trend_cci', 'trend_dpo', 'trend_kst_diff', 'trend_aroon_up', 'momentum_rsi', 'momentum_mfi', 'momentum_uo', 'momentum_stoch_signal', 'momentum_wr', 'momentum_ao', 'momentum_roc', 'others_dlr', 'others_cr', 'AO', 'MFI', 'ROC', 'RSI', 'stoch_signal', 'UO', 'WilliamsR', 'NVI', 'ATR', 'bb_bbw', 'dc_dchi', 'adx', 'adx_neg', 'aroon_up', 'CCI', 'DPO', 'KST', 'KST_diff', 'macd', 'macd_sig', 'MI', 'CR', 'DLR','label']]
#X_test = X_test[['date','adj close','high','low','open','close','volume','volume_nvi', 'volatility_atr', 'volatility_bbw', 'volatility_dchi', 'trend_macd', 'trend_macd_signal', 'trend_adx', 'trend_adx_pos', 'trend_adx_neg', 'trend_mass_index', 'trend_cci', 'trend_dpo', 'trend_kst_diff', 'trend_aroon_up', 'momentum_rsi', 'momentum_mfi', 'momentum_uo', 'momentum_stoch_signal', 'momentum_wr', 'momentum_ao', 'momentum_roc', 'others_dlr', 'others_cr', 'AO', 'MFI', 'ROC', 'RSI', 'stoch_signal', 'UO', 'WilliamsR', 'NVI', 'ATR', 'bb_bbw', 'dc_dchi', 'adx', 'adx_neg', 'aroon_up', 'CCI', 'DPO', 'KST', 'KST_diff', 'macd', 'macd_sig', 'MI', 'CR', 'DLR','label']]


#baseline3 L1+MBR
X = X[['date','adj close','high','low','open','close','volume','volume_nvi','trend_adx_pos','trend_mass_index','trend_cci','momentum_rsi','momentum_stoch','momentum_wr','RSI','Stoch','stoch_signal','TSI','WilliamsR','adx_neg','MI','CR','label']]
X_test = X_test[['date','adj close','high','low','open','close','volume','volume_nvi','trend_adx_pos','trend_mass_index','trend_cci','momentum_rsi','momentum_stoch','momentum_wr','RSI','Stoch','stoch_signal','TSI','WilliamsR','adx_neg','MI','CR','label']]
X_val = X_val[['date','adj close','high','low','open','close','volume','volume_nvi','trend_adx_pos','trend_mass_index','trend_cci','momentum_rsi','momentum_stoch','momentum_wr','RSI','Stoch','stoch_signal','TSI','WilliamsR','adx_neg','MI','CR','label']]

#X = X[['date','adj close','high','low','open','close','volume','volume_nvi','trend_adx','trend_adx_pos','trend_adx_neg','trend_mass_index','trend_cci','trend_dpo','trend_kst_diff','momentum_mfi','momentum_uo','momentum_wr','momentum_roc','others_dlr','others_cr','MFI','stoch_signal','UO','WilliamsR','NVI','adx','adx_neg','DPO','KST','KST_diff','MI','CR','DLR','label']]
#X_val = X_val[['date','adj close','high','low','open','close','volume','volume_nvi','trend_adx','trend_adx_pos','trend_adx_neg','trend_mass_index','trend_cci','trend_dpo','trend_kst_diff','momentum_mfi','momentum_uo','momentum_wr','momentum_roc','others_dlr','others_cr','MFI','stoch_signal','UO','WilliamsR','NVI','adx','adx_neg','DPO','KST','KST_diff','MI','CR','DLR','label']]
#X_test = X_test[['date','adj close','high','low','open','close','volume','volume_nvi','trend_adx','trend_adx_pos','trend_adx_neg','trend_mass_index','trend_cci','trend_dpo','trend_kst_diff','momentum_mfi','momentum_uo','momentum_wr','momentum_roc','others_dlr','others_cr','MFI','stoch_signal','UO','WilliamsR','NVI','adx','adx_neg','DPO','KST','KST_diff','MI','CR','DLR','label']]

#baseline3 L1+MBR+RFE
#X = X[['date','adj close','high','low','open','close','volume','trend_adx_pos','trend_mass_index','momentum_rsi','momentum_stoch','momentum_wr','RSI','Stoch','TSI','WilliamsR','MI','label']]
#X_test = X_test[['date','adj close','high','low','open','close','volume','trend_adx_pos','trend_mass_index','momentum_rsi','momentum_stoch','momentum_wr','RSI','Stoch','TSI','WilliamsR','MI','label']]
#X_val = X_val[['date','adj close','high','low','open','close','volume','trend_adx_pos','trend_mass_index','momentum_rsi','momentum_stoch','momentum_wr','RSI','Stoch','TSI','WilliamsR','MI','label']]
#X = X[['date','adj close','high','low','open','close','volume', 'volume_nvi', 'trend_adx', 'trend_adx_pos', 'trend_adx_neg', 'trend_mass_index', 'trend_cci', 'trend_dpo', 'trend_kst_diff', 'momentum_mfi', 'momentum_uo', 'momentum_wr', 'momentum_roc', 'others_dlr', 'others_cr', 'MFI', 'stoch_signal', 'UO', 'WilliamsR', 'NVI', 'adx', 'adx_neg', 'KST', 'KST_diff', 'MI', 'CR', 'DLR','label']]
#X_val = X_val[['date','adj close','high','low','open','close','volume', 'volume_nvi', 'trend_adx', 'trend_adx_pos', 'trend_adx_neg', 'trend_mass_index', 'trend_cci', 'trend_dpo', 'trend_kst_diff', 'momentum_mfi', 'momentum_uo', 'momentum_wr', 'momentum_roc', 'others_dlr', 'others_cr', 'MFI', 'stoch_signal', 'UO', 'WilliamsR', 'NVI', 'adx', 'adx_neg', 'KST', 'KST_diff', 'MI', 'CR', 'DLR','label']]
#X_test = X_test[['date','adj close','high','low','open','close','volume', 'volume_nvi', 'trend_adx', 'trend_adx_pos', 'trend_adx_neg', 'trend_mass_index', 'trend_cci', 'trend_dpo', 'trend_kst_diff', 'momentum_mfi', 'momentum_uo', 'momentum_wr', 'momentum_roc', 'others_dlr', 'others_cr', 'MFI', 'stoch_signal', 'UO', 'WilliamsR', 'NVI', 'adx', 'adx_neg', 'KST', 'KST_diff', 'MI', 'CR', 'DLR','label']]

#baseline3 MBR
#X = X[['date','adj close','high','low','open','close','volume','MI','trend_mass_index','CMF','volume_cmf','trend_dpo','DPO','trend_vortex_ind_neg','momentum_uo','momentum_mfi','adx_pos','label']]
#X_test = X_test[['date','adj close','high','low','open','close','volume','MI','trend_mass_index','CMF','volume_cmf','trend_dpo','DPO','trend_vortex_ind_neg','momentum_uo','momentum_mfi','adx_pos','label']]
#X = X[['date','adj close','high','low','open','close','volume','volume_adi','volume_obv','volume_cmf','volume_fi','volume_em','volume_vpt','volume_nvi','volatility_atr','trend_adx','trend_adx_pos','trend_adx_neg','trend_vortex_ind_pos','trend_vortex_ind_neg','trend_vortex_ind_diff','trend_mass_index','trend_cci','trend_dpo','trend_kst_diff','momentum_rsi','momentum_mfi','momentum_uo','momentum_stoch_signal','momentum_roc','others_cr','MFI','ROC','RSI','stoch_signal','UO','ADI','CMF','EoM','FI','NVI','OBV','VPT','adx','adx_neg','adx_pos','CCI','DPO','KST_diff','MI','vi_diff','vi_neg','vi_pos','CR','label']]
#X_val = X_val[['date','adj close','high','low','open','close','volume','volume_adi','volume_obv','volume_cmf','volume_fi','volume_em','volume_vpt','volume_nvi','volatility_atr','trend_adx','trend_adx_pos','trend_adx_neg','trend_vortex_ind_pos','trend_vortex_ind_neg','trend_vortex_ind_diff','trend_mass_index','trend_cci','trend_dpo','trend_kst_diff','momentum_rsi','momentum_mfi','momentum_uo','momentum_stoch_signal','momentum_roc','others_cr','MFI','ROC','RSI','stoch_signal','UO','ADI','CMF','EoM','FI','NVI','OBV','VPT','adx','adx_neg','adx_pos','CCI','DPO','KST_diff','MI','vi_diff','vi_neg','vi_pos','CR','label']]
#X_test = X_test[['date','adj close','high','low','open','close','volume','volume_adi','volume_obv','volume_cmf','volume_fi','volume_em','volume_vpt','volume_nvi','volatility_atr','trend_adx','trend_adx_pos','trend_adx_neg','trend_vortex_ind_pos','trend_vortex_ind_neg','trend_vortex_ind_diff','trend_mass_index','trend_cci','trend_dpo','trend_kst_diff','momentum_rsi','momentum_mfi','momentum_uo','momentum_stoch_signal','momentum_roc','others_cr','MFI','ROC','RSI','stoch_signal','UO','ADI','CMF','EoM','FI','NVI','OBV','VPT','adx','adx_neg','adx_pos','CCI','DPO','KST_diff','MI','vi_diff','vi_neg','vi_pos','CR','label']]

#baseline2
#X = X.drop(['Daily Percentage Change', 'Daily Change', 'prev adj close', 'y_t','label'], axis = 1)
#X_val = X_val.drop(['Daily Percentage Change', 'Daily Change', 'prev adj close', 'y_t'], axis = 1)
#X_test = X_test.drop(['Daily Percentage Change', 'Daily Change', 'prev adj close', 'y_t'], axis = 1)
X = X.set_index('date')
X_test = X_test.set_index('date')
X_val = X_val.set_index('date')

In [ ]:
normalized_X = X
mean = X.mean(axis = 0)
normalized_X -= mean
std = normalized_X.std(axis=0)
normalized_X /= std

normalized_X_test = X_test.astype(float)
mean = X_test.mean(axis = 0)
normalized_X_test -= mean
std = normalized_X_test.std(axis=0)
normalized_X_test /= std

normalized_X_val = X_val.astype(float)
mean = X_val.mean(axis = 0)
normalized_X_val -= mean
std = normalized_X_val.std(axis=0)
normalized_X_val /= std

normalized_X = normalized_X.drop('label', axis = 1)
norm_X = normalized_X.copy()

normalized_X_test = normalized_X_test.drop('label', axis = 1)
norm_X_test = normalized_X_test.copy()

normalized_X_val = normalized_X_val.drop('label', axis = 1)
norm_X_val = normalized_X_val.copy()


normalized_X = normalized_X.values
normalized_X_test = normalized_X_test.values
normalized_X_val = normalized_X_val.values
norm_X = norm_X.values
norm_X_test = norm_X_test.values
norm_X_val = norm_X_val.values
norm_X = np.expand_dims(norm_X, axis = 1)
print(norm_X.shape)
print(y.shape)

In [ ]:
windows = 5
lstm_input=[]
for i in range(len(norm_X)-windows):
  lstm_input.append(norm_X[i:i+windows,:])
lstm_input=np.array(lstm_input)
y = y[windows:]
lstm_input = np.squeeze(lstm_input,axis=2)
print(lstm_input.shape)
print(y.shape)

In [ ]:
norm_X_test = np.expand_dims(norm_X_test, axis = 1)
norm_X_test_input=[]
for i in range(len(norm_X_test)-windows):
  norm_X_test_input.append(norm_X_test[i:i+windows,:])
norm_X_test_input=np.array(norm_X_test_input)
print(norm_X_test_input.shape)
norm_X_test_input = np.squeeze(norm_X_test_input,axis=2)
print(norm_X_test_input.shape)
y_test = y_test[windows:]

In [ ]:
norm_X_val = np.expand_dims(norm_X_val, axis = 1)
print(norm_X_val.shape)
norm_X_val_input=[]
for i in range(len(norm_X_val)-windows):
  norm_X_val_input.append(norm_X_val[i:i+windows,:])
norm_X_val_input=np.array(norm_X_val_input)
print(norm_X_val_input.shape)
norm_X_val_input = np.squeeze(norm_X_val_input,axis=2)
print(norm_X_val_input.shape)
y_val = y_val[windows:]

In [ ]:
MAX_LENGTH = 50
EMBEDDING_DIMENSION = 512
WINDOWS = 5
BATCH_SIZE = 32

In [ ]:
def preprocessing(company, date):
    str = company + '/' + date
    filepath = os.getcwd() + '/tweet/preprocessed/' + str
    list = []
    with open(filepath, 'r') as file:
        for line in file:
            data = json.loads(line)
            list.append(data['text'])
    return list

In [ ]:
def cleaner(tweet):
    tweet = re.sub("@[A-Za-z0-9]+","",tweet) #Remove @ sign
    tweet = re.sub(r"(?:\@|http?\://|https?\://|www)\S+", "", tweet) #Remove http links
    tweet = " ".join(tweet.split())
    #tweet = tweet.replace("#", "").replace("_", "").replace("$", "") #Remove hashtag sign but keep the text
    tweet = tweet.replace("rt", "").replace("AT _ USER", "").replace("URL", "")
    words = tweet.split()
    wordsnew = tweet.split()
    for i in range(len(words)):       
        if '$' == words[i]:
         
            wordsnew = wordsnew[:i+1] + wordsnew[i + 2:]
    
    tweet = TreebankWordDetokenizer().detokenize(wordsnew)
    tweet = tweet.replace("-","").replace("$","")
    return tweet

In [ ]:
def get_data(path):
    filepath_com = os.getcwd() + '/price/test_sample/'
    company = os.listdir(filepath_com)

    total_list = []
    for i in range(len(company[:-1])):   #len(company[:-1])
        filepath_date = os.getcwd() + '/tweet/preprocessed/' + str(company[i][:-4])
        X = pd.read_csv(os.getcwd()+'/price/' + path + str(company[i]))
        date = os.listdir(filepath_date)
        
        for j in range(len(date)): #len(date)-1
            if date[j] in X['date'].values.tolist() and date[j][:-4] is not '.ipynb_checkpo':
                #print(date[j])
                list_oneday = []
                list_oneday_score = []
                list = preprocessing(company[i][:-4], date[j])
                for k in range(len(list)):
                    sentence = Sentence(TreebankWordDetokenizer().detokenize(list[k]))
                    #print(sentence)
                    #print(cleaner(str(sentence)))
                    classifier.predict(sentence)
                    
                    list_oneday_score.append(sentence.labels[0].score)
                    oneday = (embed(list[k]))
                    
                    if oneday.shape[0] >= MAX_LENGTH:
                        list_oneday.append(oneday[0:MAX_LENGTH])
                    else:
                        one_as_vector = tf.reshape(oneday,[-1])
                        zero_paddings = tf.zeros([MAX_LENGTH*EMBEDDING_DIMENSION]-tf.shape(one_as_vector),dtype=oneday.dtype)
                        oneday_padded = tf.concat([one_as_vector, zero_paddings], 0)
                        results = tf.reshape(oneday_padded, [MAX_LENGTH, EMBEDDING_DIMENSION])
   
           
            
                        list_oneday.append(results)
                
                list_oneday_score = np.array(list_oneday_score)
                list_oneday_score = list_oneday_score/sum(list_oneday_score)
                res = list_oneday * list_oneday_score[:, np.newaxis, np.newaxis]
                #list_oneday = list_oneday*list_oneday_score
                list_oneday = tf.convert_to_tensor(res)
                #print(list_oneday)
                
               
                list_oneday = np.mean(list_oneday,axis=0)
                list_oneday = tf.convert_to_tensor(list_oneday)
                
                total_list.append(list_oneday)
        #print(len(total_list))
    total_list = tf.convert_to_tensor(total_list)
    #gru = tf.keras.layers.GRU(64)
    #total_list = (gru(total_list))
    print(total_list.shape)
    #np.save('embeddings.npy', embed(list[0]))
    #all_embeddings = np.load('embeddings.npy')
    #print(torch.FloatTensor(np.asarray(all_embeddings)))
    return total_list

In [ ]:
training_datat = get_data('train_sample/')
validation_datat = get_data('val_sample/')
testing_datat = get_data('test_sample/')

In [ ]:
xtrain_inputt=[]
for i in range(len(training_datat)-WINDOWS):
  xtrain_inputt.append(training_datat[i:i+WINDOWS,:])
xtrain_inputt=np.array(xtrain_inputt)
print(xtrain_inputt.shape)
xtrain_inputt = xtrain_inputt.reshape(15222, WINDOWS, -1)

xval_inputt=[]
for i in range(len(validation_datat)-WINDOWS):
  xval_inputt.append(validation_datat[i:i+WINDOWS,:])
xval_inputt=np.array(xval_inputt)
print(xval_inputt.shape)
xval_inputt = xval_inputt.reshape(1654, WINDOWS, -1)

xtest_inputt=[]
for i in range(len(testing_datat)-WINDOWS):
  xtest_inputt.append(testing_datat[i:i+WINDOWS,:])
xtest_inputt=np.array(xtest_inputt)
print(xtest_inputt.shape)
xtest_inputt = xtest_inputt.reshape(2513, WINDOWS, -1)

In [ ]:
from keras import backend as K

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall
    #return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

def mcc_m(y_true, y_pred):
    tp = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    tn = K.sum(K.round(K.clip((1 - y_true) * (1 - y_pred), 0, 1)))
    fp = K.sum(K.round(K.clip((1 - y_true) * y_pred, 0, 1)))
    fn = K.sum(K.round(K.clip(y_true * (1 - y_pred), 0, 1)))

    num = tp * tn - fp * fn
    den = (tp + fp) * (tp + fn) * (tn + fp) * (tn + fn)
    return num / K.sqrt(den + K.epsilon())

In [ ]:
def custom_loss(y_true, y_pred):
 
   # bce_loss = tf.keras.losses.binary_crossentropy(y_true, y_pred, from_logits=False)
    bce = tf.keras.losses.BinaryCrossentropy(from_logits=False, label_smoothing=0.1)
    bce_loss = bce(y_true, y_pred)
    mse_loss = tf.keras.losses.mean_squared_error(y_true, y_pred)
    loss = 0.85*bce_loss + 0.15*mse_loss
    
    return bce_loss

In [ ]:
def create_model(input_shape, input_shapet):
    inputs = Input(shape=input_shape)
    inputst = Input(shape=input_shapet)
    x = TCN(nb_filters=16, return_sequences=True)(inputs)
    x = LSTM(64, input_shape=(WINDOWS,21), return_sequences=True)(x)
    x = Dropout(0.3)(x)
    
    for i in range(3):
        x1 = TCN(nb_filters=16, return_sequences=True)(inputs)
        x1 = LSTM(64, input_shape=(WINDOWS,21), return_sequences=True)(x1)
        x1 = Dropout(0.3)(x1)
        x1 = LSTM(32, return_sequences=True)(x1)
        x1 = Dropout(0.3)(x1)
        x1 = LSTM(16, return_sequences=True)(x1)
        x1 = Dropout(0.3)(x1)
        x = concatenate([x,x1],axis=-1)
    
    x = SeqSelfAttention(attention_activation='tanh')(x)
    x = LSTM(16, return_sequences=False)(x)
    x = Dropout(0.3)(x)
    x = Dense(16,kernel_initializer="uniform",activation='relu')(x)
    
    xt = TCN(nb_filters=16, return_sequences=True)(inputst)
    xt = Bidirectional(GRU(16, return_sequences=True))(xt)
    xt = Dropout(0.3)(xt)
    xt = Dense(16,kernel_initializer="uniform",activation='relu')(xt)
    
    
    for i in range(2):
        x2 = TCN(nb_filters=16, return_sequences=True)(inputst)
        x2 = Bidirectional(GRU(16, return_sequences=True))(x2)
        x2 = Dropout(0.3)(x2)
        
      #  x2 = Dense(16,kernel_initializer="uniform",activation='relu')(inputst)
        xt = concatenate([xt,x2],axis=-1)
        #x2 = Dense(16,kernel_initializer="uniform",activation='relu')(xt)
    
    xt = Bidirectional(GRU(16, return_sequences=False))(xt)
    xt = Dropout(0.3)(xt)
    xt = Dense(8,kernel_initializer="uniform",activation='relu')(xt)
    x_all = concatenate([x, xt],axis=-1)
  #  x_all = Dense(16,kernel_initializer="uniform",activation='relu')(x_all)
    x_final = Dense(1,kernel_initializer="uniform",activation='relu')(x_all)
    
    model = Model(inputs=[inputs,inputst], outputs = x_final)

    return model

In [ ]:
model = create_model(input_shape=lstm_input.shape[1:], input_shapet=xtrain_inputt.shape[1:])
model.compile(loss='binary_crossentropy',optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.0005), metrics=['accuracy', f1_m, mcc_m])

In [ ]:
my_callbacks = [
    # keras.callbacks.EarlyStopping(patience=2),
    keras.callbacks.ModelCheckpoint(filepath='modellstm.{epoch:02d}.h5'),
    keras.callbacks.TensorBoard(log_dir='./logs'),
]

history=model.fit(
    [lstm_input,xtrain_inputt],
    y,
    batch_size=32,
    epochs=100,
    validation_data=([norm_X_val_input,xval_inputt], y_val),
    verbose=1,
    callbacks=my_callbacks)

In [ ]:
import matplotlib.pyplot as plt
plt.plot(history.history['loss'], label='train')
plt.plot(history.history['val_loss'], label='validation')
plt.legend()
plt.show()

In [ ]:
plt.plot(history.history['accuracy'], label='train')
plt.plot(history.history['val_accuracy'], label='validation')
plt.legend()
plt.show()

In [ ]:
test_acc = model.evaluate([norm_X_test_input,xtest_inputt], y_test, batch_size = 32)
#y_pred = model.predict_classes(norm_X_test_input,batch_size = 32)

In [ ]:
#@tf.autograph.experimental.do_not_convert
acc_hist = []
n = 1
for i in range(100):
  k = str(n).zfill(2)
  n += 1
  name = 'modellstm.' + k +'.h5'
  model11 = load_model(name, custom_objects={'SeqSelfAttention':SeqSelfAttention.get_custom_objects()['SeqSelfAttention'],'TCN':TCN,'f1_m':f1_m,'mcc_m':mcc_m,'custom_loss':custom_loss,'optim':optim})
  model11.compile(loss='binary_crossentropy',optimizer='rmsprop', metrics=['accuracy', f1_m, mcc_m])
  loss,acc,f1,mcc = model11.evaluate([norm_X_test_input,xtest_inputt],y_test,batch_size=32)
  acc_hist.append(acc)
  print(acc)
 
    
print(max(acc_hist))
print(acc_hist.index(max(acc_hist))+1)

In [ ]:
name = 'modellstm.58.h5'
model11 = load_model(name, custom_objects={'SeqSelfAttention':SeqSelfAttention.get_custom_objects()['SeqSelfAttention'],'TCN':TCN,'f1_m':f1_m,'mcc_m':mcc_m,'custom_loss':custom_loss,'optim':optim})
model11.compile(loss='binary_crossentropy',optimizer='rmsprop', metrics=['accuracy', f1_m, mcc_m])
loss,acc,f1,mcc = model11.evaluate([norm_X_test_input,xtest_inputt],y_test,batch_size=32)
print(acc)
print(f1)
print(mcc)